##🚗 자동차 디자인 생성 및 변형 실습 (Stable Diffusion + ControlNet)

이 노트북은 텍스트 또는 스케치 이미지 기반으로 자동차 디자인을 생성 및 변형하는 실습을 제공합니다.

Stable Diffusion v1.5 및 ControlNet(Lineart)을 활용합니다.

###🧠 Stable Diffusion이란?

Stable Diffusion은 텍스트를 기반으로 이미지를 생성하는 Text-to-Image 생성형 AI 모델로, 2022년 Stability AI에서 공개한 오픈소스 딥러닝 모델입니다.

| 개념                | 설명                                       |
| ----------------- | ---------------------------------------- |
| **Diffusion**     | 무작위한 노이즈 이미지에서 점점 정보를 복원해가는 방식 (역확산 과정)  |
| **Latent Space**  | 실제 고해상도 이미지 공간 대신, 압축된 잠재 공간에서 학습과 생성 진행 |
| **CLIP**          | 텍스트와 이미지를 연결해주는 사전 학습된 언어-비전 모델          |
| **U-Net**         | 이미지에 점진적으로 노이즈를 제거하는 네트워크 (생성 핵심 모듈)     |
| **VAE (인코더/디코더)** | 이미지를 잠재 공간(latent)으로 인코딩하고, 다시 이미지로 복원   |

###🔁 작동 원리 (간단 순서)
1. 입력 텍스트 프롬프트(ex: "a futuristic electric car")를 CLIP으로 임베딩

2. 랜덤한 노이즈 이미지를 latent space에 생성

3. U-Net이 점차적으로 노이즈를 제거 → 이미지로 복원

4. 디코더(VAE)가 latent 이미지 → 실제 이미지로 변환

🧩 이렇게 해서 "텍스트 → 이미지" 변환이 가능해집니다.

###🛠️ 주요 특징

| 항목         | 내용                                                         |
| ---------- | ---------------------------------------------------------- |
| **모델 크기**  | 약 4GB (`v1.4`, `v1.5`)                                     |
| **출력 해상도** | 기본 512x512, 확장 가능                                          |
| **학습 데이터** | LAION-5B, 수십억 개의 텍스트-이미지 쌍                                 |
| **사용 방식**  | Hugging Face `diffusers`, InvokeAI, AUTOMATIC1111, Colab 등 |
| **라이선스**   | 오픈소스 (CreativeML Open RAIL-M)                              |


###🎯 활용 사례

| 분야     | 활용 예                     |
| ------ | ------------------------ |
| 디자인    | 제품 시안, 패션, 인테리어, 자동차 콘셉트 |
| 게임     | 캐릭터 생성, 배경 디자인           |
| 교육/연구  | 과학 시각화, 데이터 기반 표현        |
| 마케팅    | 광고 시각 자료, 소셜 콘텐츠 자동화     |
| 엔터테인먼트 | 스토리보드, NFT 아트 생성         |

###🧪 확장 모델들

- ControlNet: 입력 이미지의 구조(Lineart, Depth 등)를 유지하면서 생성 제어

- DreamBooth: 특정 인물/브랜드 스타일로 미세 조정

- Inpainting / Outpainting: 이미지 일부만 변경하거나 확장


###✅ 1. 환경 설정

In [ ]:
!pip install diffusers transformers accelerate controlnet_aux opencv-python gradio xformers --quiet

###✅ 2. Hugging Face 로그인

In [ ]:
from huggingface_hub import login
login(token="여러분의 token을 넣으시면 됩니다.")

###✅ 3. 모델 및 파이프라인 로딩

In [ ]:
from diffusers import StableDiffusionControlNetPipeline, ControlNetModel
import torch

# ControlNet 모델 올바른 이름 사용
controlnet = ControlNetModel.from_pretrained(
    "lllyasviel/control_v11p_sd15_lineart", torch_dtype=torch.float16
)

pipe = StableDiffusionControlNetPipeline.from_pretrained(
    "runwayml/stable-diffusion-v1-5",
    controlnet=controlnet,
    torch_dtype=torch.float16
).to("cuda")



###✅ 4. 참고 이미지 없이 프롬프트 만으로 생성

In [ ]:
from diffusers import StableDiffusionPipeline
import torch

# 모델 로딩 (v1.5 버전 기준, Hugging Face 로그인 필요)
pipe = StableDiffusionPipeline.from_pretrained(
    "runwayml/stable-diffusion-v1-5",
    torch_dtype=torch.float16
).to("cuda")

# 프롬프트 예시
prompt = "a futuristic electric SUV concept car, cyberpunk style, high detail, photorealistic, studio lighting"

# 이미지 생성
image = pipe(prompt=prompt, num_inference_steps=40, guidance_scale=7.5).images[0]

# 출력
image

###✅ 5. 입력 이미지가 주어졌을때 생성

In [ ]:
from PIL import Image

# 로컬에 저장된 이미지 경로
image_path = 'car_sketch.jpg'

# 이미지 불러오기
input_image = Image.open(image_path).convert("RGB")

prompt = 'a futuristic electric SUV car design, silver tone, aerodynamic shape, concept car'

input_image
# input_image.show()

✅ 6. 이미지 생성

In [ ]:
output = pipe(prompt=prompt, image=input_image, num_inference_steps=30)
generated_image = output.images[0]
generated_image.save('car_design_generated.png')
generated_image

##🚀 추가 실습 아이디어 + 실습 코드

###✅ 1. 프롬프트 변경 실험

In [ ]:
# 현재 크기 확인
w, h = input_image.size

# 줄이고 싶은 목표 너비 지정 (예: 512px)
target_width = 512
aspect_ratio = h / w
target_height = int(target_width * aspect_ratio)

# 크기 조정
init_image = input_image.resize((target_width, target_height))
input_image.size

In [ ]:
prompt = "pencil sketch of a retro car, 1960s, side view"

output = pipe(prompt=prompt, image=init_image, num_inference_steps=30)
output_image = output.images[0]
output_image

### ✅ 2. 배경/스타일 조건 추가 실험

In [ ]:
prompt = "a modern electric SUV on a mountain road at sunset, photorealistic, 4k"
output = pipe(prompt=prompt, image=init_image, num_inference_steps=40)
output.images[0]

###✅ 3. Inpainting 실습 코드 (마스크 영역만 변경)

💡 설명:

- 이미지에서 일부를 마스크로 지정한 후, 해당 영역만 재생성

- 예: 자동차 뒷부분만 바꾸기

📦 필요한 라이브러리 설치

In [ ]:
!pip install diffusers transformers accelerate xformers opencv-python --quiet

🧪 실습 코드

In [ ]:
from diffusers import StableDiffusionInpaintPipeline
from PIL import Image
import torch
import numpy as np
import requests
from io import BytesIO

# ✅ 모델 로딩
pipe = StableDiffusionInpaintPipeline.from_pretrained(
    "stabilityai/stable-diffusion-2-inpainting",
    torch_dtype=torch.float16
).to("cuda")

# 마스크 생성 (중간 직사각형 흰색 마스킹)
mask = np.zeros((512, 512), dtype=np.uint8)
mask[200:320, 150:360] = 255  # (top:bottom, left:right)

mask_image = Image.fromarray(mask).convert("RGB")

# ✅ 프롬프트 설정
prompt = "replace the masked part with a futuristic electric engine and glowing light tubes"

# ✅ Inpainting 실행
output = pipe(prompt=prompt, image=init_image, mask_image=mask_image).images[0]
output


##🚗 AI로 자동차 부품 설계 아이디어 생성 실습

###✅ 1. 라이브러리 설치

In [ ]:
!pip install diffusers transformers accelerate xformers --quiet

###✅ 2. 모델 로딩

In [ ]:
from diffusers import StableDiffusionPipeline
import torch

model_id = "runwayml/stable-diffusion-v1-5"
pipe = StableDiffusionPipeline.from_pretrained(
    model_id,
    torch_dtype=torch.float16
).to("cuda")


###✏️ Step 1: 프롬프트 작성

In [ ]:
# 전기차 휠 프롬프트 예시
prompt = "a lightweight carbon fiber wheel for electric vehicle, futuristic aerodynamic design, CAD rendering style"
print(prompt)

### 🎨 Step 2: 이미지 생성

In [ ]:
image = pipe(prompt=prompt, num_inference_steps=40, guidance_scale=7.5).images[0]
image.save("ev_wheel_design.png")
image

###🔄 Step 3: 설계 조건 변경 실험

In [ ]:
prompt2 = "spoke-type 5-arm aluminum alloy wheel for a high-performance sports EV, ultra-futuristic, detailed"
image2 = pipe(prompt=prompt2, num_inference_steps=40, guidance_scale=7.5).images[0]
image2.save("ev_sports_wheel.png")
image2

###✅ CAD 연계 아이디어: Depth → Mesh

🧠 dpt-hybrid-midas란?

dpt-hybrid-midas는 DPT (Dense Prediction Transformer) 구조 기반의 사전 학습된 단일 이미지 깊이 추정 모델입니다.

- 전체 모델 이름: Intel/dpt-hybrid-midas

- 제공: Intel Labs

- 플랫폼: Hugging Face Transformers

- 목적: 2D 이미지에서 픽셀 단위의 거리 정보(깊이)를 예측

📐 어디에 쓰이나요?

- 3D 재구성

- 로봇의 거리 인식

- AR/VR의 공간 이해

- CAD/메시 추출 전처리

- 자동차 부품 디자인 → 3D 모델 연동용 depth 추출

🔧 모델 특징 요약

| 항목     | 설명                                  |
| ------ | ----------------------------------- |
| 구조     | Vision Transformer (ViT) 기반의 DPT 모델 |
| 입력     | RGB 이미지 384x384 또는 512x512          |
| 출력     | 픽셀당 상대적 깊이값 (단위 없음)                 |
| 학습 데이터 | 다양한 거리 라벨이 포함된 RGB 이미지              |
| 장점     | 경량 모델보다 훨씬 정밀한 구조 추정 가능             |


In [ ]:
# 예시: depth map 생성용 모델 불러오기
from transformers import DPTImageProcessor, DPTForDepthEstimation

depth_model = DPTForDepthEstimation.from_pretrained("Intel/dpt-hybrid-midas")
processor = DPTImageProcessor.from_pretrained("Intel/dpt-hybrid-midas")

# 이미지 depth 변환
inputs = processor(images=init_image, return_tensors="pt")
with torch.no_grad():
    outputs = depth_model(**inputs)
    predicted_depth = outputs.predicted_depth

# 정규화 후 시각화
import matplotlib.pyplot as plt
depth = predicted_depth.squeeze().numpy()
depth = (depth - depth.min()) / (depth.max() - depth.min())

plt.imshow(depth, cmap="viridis")
plt.axis("off")
plt.title("Predicted Depth (for mesh or CAD)")
plt.show()


##🚘 차량 내부 디자인 시각화 실습

###✏️ Step 1: 기본 프롬프트 작성

In [ ]:
prompt = "futuristic electric car interior, ambient lighting, panoramic roof, minimal dashboard, high detail, soft materials, 3D render"
print(prompt)

###🎨 Step 2: 디자인 이미지 생성

In [ ]:
image = pipe(prompt=prompt, num_inference_steps=40, guidance_scale=7.5).images[0]
image.save("car_interior_futuristic.png")
image

###🔄 Step 3: 디자인 변형 실험

⏹ 예시 1 – 클래식 스타일

In [ ]:
prompt_classic = "retro style EV interior, analog gauges, wood finish dashboard, leather seats, soft lighting"
image2 = pipe(prompt=prompt_classic, num_inference_steps=40, guidance_scale=7.5).images[0]
image2.save("car_interior_retro.png")
image2

⏹ 예시 2 – 패밀리형 인테리어

In [ ]:
prompt_family = "family-friendly electric car interior, spacious second row, child seat, wide windows, calm lighting"
image3 = pipe(prompt=prompt_family, num_inference_steps=40, guidance_scale=7.5).images[0]
image3.save("car_interior_family.png")
image3


⏹ 예시 2 – 패밀리형 인테리어


In [ ]:
prompt_family = "family-friendly electric car interior, spacious second row, child seat, wide windows, calm lighting"
image3 = pipe(prompt=prompt_family, num_inference_steps=40, guidance_scale=7.5).images[0]
image3.save("car_interior_family.png")
image3

###🧩 확장 실습 1: ControlNet(Lineart)으로 구조 고정, 스타일만 변경

In [ ]:
from diffusers import StableDiffusionControlNetPipeline, ControlNetModel
from diffusers.utils import load_image
import torch
from PIL import Image
import requests
from io import BytesIO

# ✅ ControlNet 모델 불러오기
controlnet = ControlNetModel.from_pretrained(
    "lllyasviel/control_v11p_sd15_lineart", torch_dtype=torch.float16
)
pipe = StableDiffusionControlNetPipeline.from_pretrained(
    "runwayml/stable-diffusion-v1-5",
    controlnet=controlnet,
    torch_dtype=torch.float16
).to("cuda")

# ✅ 스타일 프롬프트 정의
prompt_style = "a luxury car interior with ambient lighting and wooden finish, 3D render"

# ✅ 이미지 생성 (형상은 고정, 스타일만 변경됨)
output = pipe(prompt=prompt_style, image=init_image, num_inference_steps=30)
output.images[0]


###🌐 확장 실습 2: Depth map 추출 → Blender 연동용 displacement map

In [ ]:
from transformers import DPTImageProcessor, DPTForDepthEstimation
import matplotlib.pyplot as plt

depth_model = DPTForDepthEstimation.from_pretrained("Intel/dpt-hybrid-midas")
processor = DPTImageProcessor.from_pretrained("Intel/dpt-hybrid-midas")

# 이미지 → Depth 예측
inputs = processor(images=output.images[0], return_tensors="pt")
with torch.no_grad():
    outputs = depth_model(**inputs)
    depth_map = outputs.predicted_depth.squeeze().numpy()

# 정규화 후 저장 (Blender에서 높이 맵으로 사용 가능)
depth_norm = (depth_map - depth_map.min()) / (depth_map.max() - depth_map.min())
plt.imsave("depth_for_blender.png", depth_norm, cmap="gray")

plt.imshow(depth_norm, cmap="plasma")
plt.title("Depth Map for Blender")
plt.axis("off")
plt.show()